In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [2]:
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")

In [3]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

categories = ["Phone", "Phone Accessories", "Tablets", "Wearable Devices", "Stationary", "Software", "Computers", "Laptops", "Computer Accessories", "Gaming", "Television", "Headphones", "Speakers", "Cameras", "Musical", "Appliances", "Clothing", "Footwear", "Kitchen", "Furniture", "Art Supplies","Pet Supplies", "Sports", "Fitness", "Luggage", "Toys", "Baby Products", "Car Accessories", "Bike Accessories", "Industrial Supplies", "Books", "Movies", "Skincare"]

def categorize_product_advanced(product_name):
    result = classifier(product_name, categories)
    return result["labels"][0]

In [4]:
def scrape_products(browser):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    page_data = []
    for s in soup.find_all(attrs={"role": "listitem"}):
        try:
            name = s.find("h2").get("aria-label")
            rating = s.find(class_="a-icon-alt").text[:3]
            number = s.find(attrs={"data-cy":"reviews-block"}).find(class_="a-link-normal s-underline-text s-underline-link-text s-link-style").get("aria-label")
            number = number[:-8]
            price = s.find(class_="a-price-whole").text
            category = categorize_product_advanced(name)
            page_data.append([name, rating, number, price, category])
        except AttributeError:
            continue
    return page_data

In [5]:
browser = webdriver.Chrome(options=options)
data = []

try:
    browser.get('https://www.amazon.in/s?me=A2TZPEKUT2134H&page=1')
    time.sleep(5)
    print("Scraping page 1...")
    data = scrape_products(browser)
finally:
    browser.quit()

Scraping page 1...


In [6]:
data

[['Medicube Collagen Overnight Wrapping Peel Off Facial Mask Pack - Elasticity & Hydration Care, Reduces Sagging & Dullness - Hydrolyzed Collagen For Glowing Skin - Korean Skin Care, 2.53 fl.oz',
  '4.2',
  '2,510',
  '3,330',
  'Skincare'],
 ['Pentel Ain Stein 0.5Mm Mechanical Pencil Lead | Lead of Grade 4B | Smooth & Not Scratchy | Easy to Insert Inside The Pencil | Pack of 40 Pcs (C275-4B), Black',
  '4.5',
  '8,900',
  '289',
  'Stationary'],
 ['MAGICYOYO Magic Yoyo N11 Alloy Aluminum Professional Yoyo Toy (Black With Golden)',
  '4.3',
  '5,844',
  '2,899',
  'Toys'],
 ['Genki: An Integrated Course in Elementary Japanese 1 [3rd Edition]',
  '4.8',
  '2,818',
  '4,994',
  'Speakers'],
 ['SAPHIR (Saphir) [Saphir] SAPHIR Cream Bead Wax Polish 50 ml',
  '4.6',
  '2,305',
  '1,611',
  'Skincare'],
 ['LIHIT LAB Compact Pen Case (Pencil Case), Water & Stain Repellent,3.5" x 6.5\'\', Navy (A7687-11)',
  '4.5',
  '1,511',
  '1,079',
  'Stationary'],
 ['LEGO Technic Off-Road Race Buggy Car 

In [7]:
# browser = webdriver.Chrome(options=options)
# data=[]

# try:
#     browser.get('https://www.amazon.in/s?me=A2TZPEKUT2134H&page=1')
#     time.sleep(5)
#     page_number = 1
#     while True:
#         print(f"Scraping page {page_number}...")
#         products = scrape_products(browser)
#         if not products:
#             print("No more products found.")
#             break
#         data.extend(products)
#         page_number += 1
#         next_url = f'https://www.amazon.in/s?me=A2TZPEKUT2134H&page={page_number}'
#         browser.get(next_url)
#         time.sleep(5)
# finally:
#     browser.quit()

In [12]:
# import joblib

# vectorizer = joblib.load("vectorizer.pkl")
# model = joblib.load("nb_model.pkl")

# def predict_category(title):
#     title_tfidf = vectorizer.transform([title])
#     predicted_category = model.predict(title_tfidf)[0]
#     return predicted_category

In [13]:
# new_title = "LIHITLAB Zipper Pen Case, 7.9 × 2 × 4.7 Inches, Black (A7551-24)"
# predicted_category = predict_category(new_title)
# print(f"Predicted Category: {predicted_category}")